In [1]:
%pip install -q --upgrade langchain langchain-openai langchain-core langchain_community docx2txt pypdf  langchain_chroma sentence_transformers

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### What is Retrieval Augmented Generation (RAG)?
RAG is a technique that enhances language models by combining them with a retrieval system. It allows the model to access and utilize external knowledge when generating responses.

The process typically involves:
#### Indexing a large corpus of documents

In [2]:
import langchain
print(langchain.__version__)

0.3.8


In [ ]:
import os
os.environ["TOGETHER_API_KEY"] = ""

In [4]:
#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_API_KEY"] = "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"
#os.environ["LANGCHAIN_PROJECT"] = "Multilingual-RAG"

### Call LLM

In [5]:
from langchain_openai import ChatOpenAI
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="meta-llama/Llama-Vision-Free",
)


llm_response = llm.invoke("Tell me a joke")

llm_response

/Users/abubakarsiddique/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


AIMessage(content="Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 14, 'total_tokens': 32, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'meta-llama/Llama-Vision-Free', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-96a72051-d219-4440-819a-6f9274c4ce14-0', usage_metadata={'input_tokens': 14, 'output_tokens': 18, 'total_tokens': 32, 'input_token_details': {}, 'output_token_details': {}})

### Parsing Output

In [6]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(llm_response)

"Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta!"

### Simple Chain

In [7]:
chain = llm | output_parser
chain.invoke("Tell me a joke")

'A man walks into a library and asks the librarian, "Do you have any books on Pavlov\'s dogs and Schrödinger\'s cat?" The librarian replies, "It rings a bell, but I\'m not sure if it\'s here or not."'

In [8]:
#from typing import List
#from pydantic import BaseModel, Field # type: ignore

#class MobileReview(BaseModel):
#    phone_model: str = Field(description="Name and model of the phone")
#    rating: float = Field(description="Overall rating out of 5")
#    pros: List[str] = Field(description="List of positive aspects")
#    cons: List[str] = Field(description="List of negative aspects")
#    summary: str = Field(description="Brief summary of the review")

#review_text = """
#Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous,
#colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
#stronger. Battery life's solid, lasts me all day no problem.

#Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
#Also, still getting used to the new button layout, keep hitting Bixby by mistake.

#Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
#being perfect. If you're due for an upgrade, definitely worth checking out!
#"""

#structured_llm = llm.with_structured_output(MobileReview)
#output = structured_llm.invoke(review_text)
#output

In [9]:
#output.pros

### Prompt Template

In [10]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
prompt.invoke({"topic": "snake"})

ChatPromptValue(messages=[HumanMessage(content='Tell me a short joke about snake', additional_kwargs={}, response_metadata={})])

In [11]:
chain = prompt | llm | output_parser
chain.invoke({"topic": "snake"})

'Why did the snake visit the doctor?\n\nBecause it had a hiss-terical pain!'

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define the prompt
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

# Initialize the LLM
llm = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="meta-llama/Llama-Vision-Free",
)

# Define the output parser
output_parser = StrOutputParser()

# Compose the chain
chain = prompt | llm | output_parser

# Use the chain
result = chain.invoke({"topic": "Lion"})
print(result)

Why did the lion go to the dentist?

Because it had a ROAR-ing toothache!


### LLM Messages

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage(content="You are a helpful assistant that tells jokes.")
human_message = HumanMessage(content="Tell me about Cakes")
llm.invoke([system_message, human_message])

AIMessage(content="Cakes! The sweetest topic around!\n\nWhy did the cake go to therapy? Because it was feeling crumby!\n\nBut seriously, cakes are amazing creations that bring people together. From classic vanilla to decadent chocolate, there's a cake to suit every taste. And let's not forget the frosting – the icing on the cake (get it?)!\n\nDid you know that cakes have been around since ancient Egypt? They were made with honey and nuts, and were often served at special occasions. Who knew our ancestors loved cake parties too?\n\nWhat's your favorite type of cake? Do you have a sweet tooth for something specific?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 128, 'prompt_tokens': 29, 'total_tokens': 157, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'meta-llama/Llama-Vision-Free', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b992f954-8132-4403-a3fb-9669111760c9-0', us

In [14]:
template = ChatPromptTemplate([
    ("system", "You are a helpful assistant that tells jokes."),
    ("human", "Tell me about {topic}")
])

prompt_value = template.invoke(
    {
        "topic": "Cakes"
    }
)
prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant that tells jokes.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about Cakes', additional_kwargs={}, response_metadata={})])

In [15]:
llm.invoke(prompt_value)

AIMessage(content="Cakes! They're the icing on the cake (get it?)! But seriously, cakes are a sweet treat that brings people together. Did you hear about the cake that went to therapy? It was feeling crumby!\n\nBut let's get serious for a sec. Cakes have a rich history, with ancient Egyptians and Greeks baking cakes for special occasions. And today, we have all sorts of delicious cakes, from classic vanilla and chocolate to creative flavors like red velvet and carrot cake.\n\nSome fun facts about cakes:\n\n* The world's largest cake was over 2,000 pounds and had 2,000 candles!\n* The average American eats around 19 pounds of cake per year. That's a lot of frosting!\n* Cakes can be used for more than just dessert – they can be used as a base for science experiments, like making a volcano erupt!\n\nOkay, okay, I'll stop with the cake puns now. But before I go, here's one more: Why did the cake go to the party? Because it was a batter guest!\n\nHope that made you laugh and satisfied your 

In [16]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from typing import List
from langchain_core.documents import Document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len
)

docx_loader = Docx2txtLoader("./docs/NextView_Technologies_Profile_Urdu.docx")
documents = docx_loader.load()

print(len(documents))

splits = text_splitter.split_documents(documents)

print(f"Split the documents into {len(splits)} chunks.")

1
Split the documents into 5 chunks.


In [17]:
documents[0]

Document(metadata={'source': './docs/NextView_Technologies_Profile_Urdu.docx'}, page_content='کمپنی کا نام: نیکسٹ ویو ٹیکنالوجیز\nقیام: 2010\nمرکز: لاہور، پاکستان\n\nہمارے بارے میں:\nنیکسٹ ویو ٹیکنالوجیز پاکستان کی ایک معروف ٹیکنالوجی کمپنی ہے جو جدید سافٹ ویئر اور ڈیجیٹل حل فراہم کرتی ہے۔ \nہماری ٹیم کا مقصد کاروباروں کو ڈیجیٹل دور میں ترقی دینے کے لیے تخلیقی اور جدید ٹیکنالوجی پر مبنی خدمات فراہم کرنا ہے۔\n\nاہم خدمات:\n1. ویب اور موبائل ایپلیکیشن ڈویلپمنٹ\n2. ڈیجیٹل مارکیٹنگ اور برانڈنگ\n3. ای کامرس سلوشنز\n4. کلاؤڈ بیسڈ ایپلیکیشنز\n5. آرٹیفیشل انٹیلیجنس پر مبنی پراڈکٹس\n\nاہم کلائنٹس:\n1. معروف ریٹیل اسٹورز نیٹ ورک\n2. تعلیمی ادارے اور یونیورسٹیز\n3. فیشن اور ٹیکسٹائل انڈسٹری\n4. صحت کے شعبے میں خدمات فراہم کرنے والی تنظیمیں\n\nایوارڈز اور اعزازات:\n- بہترین ڈیجیٹل سروسز کمپنی 2021\n- آئی ٹی انوویشن ایوارڈ 2020\n- پاکستان کی ٹاپ 100 ٹیکنالوجی کمپنیز میں شامل\n\nرابطے کی معلومات:\nویب سائٹ: www.nextviewtech.com\nای میل: info@nextviewtech.com\nفون: +92 (42) 34567890\nپتہ: 456 ڈیجیٹل 

In [18]:
splits[1]

Document(metadata={'source': './docs/NextView_Technologies_Profile_Urdu.docx'}, page_content='ہمارے بارے میں:\nنیکسٹ ویو ٹیکنالوجیز پاکستان کی ایک معروف ٹیکنالوجی کمپنی ہے جو جدید سافٹ ویئر اور ڈیجیٹل حل فراہم کرتی ہے۔ \nہماری ٹیم کا مقصد کاروباروں کو ڈیجیٹل دور میں ترقی دینے کے لیے تخلیقی اور جدید ٹیکنالوجی پر مبنی خدمات فراہم کرنا ہے۔')

In [19]:
splits[0].metadata

{'source': './docs/NextView_Technologies_Profile_Urdu.docx'}

In [20]:
splits[0].page_content

'کمپنی کا نام: نیکسٹ ویو ٹیکنالوجیز\nقیام: 2010\nمرکز: لاہور، پاکستان'

In [21]:
# 1. Function to load documents from a folder

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        elif filename.endswith('.txt'):
            loader = TextLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

# Load documents from a folder
folder_path = "./docs"
documents = load_documents(folder_path)

print(f"Loaded {len(documents)} documents from the folder.")
splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")

Unsupported file type: .DS_Store
Loaded 2 documents from the folder.
Split the documents into 12 chunks.


In [ ]:
from langchain_community.embeddings import JinaEmbeddings
embeddings = JinaEmbeddings(
    jina_api_key="", model_name="jina-embeddings-v3"
)



# 4. Embedding Documents

document_embeddings = embeddings.embed_documents([split.page_content for split in splits])

print(f"Created embeddings for {len(document_embeddings)} document chunks.")

Created embeddings for 12 document chunks.


In [23]:
document_embeddings[0]

[0.039562646,
 -0.1678835,
 -0.033479914,
 0.07796331,
 0.020988848,
 0.033447348,
 -0.073860295,
 0.06723421,
 -0.03621315,
 0.0048393896,
 0.064838976,
 0.10490065,
 0.06885033,
 0.025702339,
 -0.019857733,
 -0.095474094,
 -0.16654961,
 0.0942608,
 0.038274907,
 0.05775096,
 -0.106425114,
 -0.022127878,
 0.035077944,
 0.13666102,
 -0.015386192,
 0.115907155,
 0.059515428,
 0.005777327,
 -0.02542826,
 0.04835877,
 0.07165442,
 0.039955247,
 -0.05186961,
 -0.0011829938,
 0.048365403,
 0.022405121,
 -0.054259714,
 0.09549338,
 -0.035215132,
 0.038746472,
 0.051126674,
 0.09054131,
 0.022253497,
 0.0047408035,
 -0.016624816,
 -0.017458316,
 0.043613832,
 -0.020425977,
 -0.03665984,
 -0.010318311,
 0.000610106,
 -0.022059584,
 -0.028088525,
 -0.0230158,
 0.06504521,
 0.033805702,
 -0.027268255,
 -0.05539944,
 0.039920423,
 -0.004556493,
 -0.006776171,
 0.022336846,
 -0.034616493,
 0.005975196,
 0.041322917,
 0.08851151,
 0.042197313,
 -0.041517396,
 0.019787066,
 -0.019470876,
 -0.0299953

### Create and persist Chroma vector store

In [ ]:
from langchain_chroma import Chroma

embedding_function = JinaEmbeddings(
    jina_api_key="", model_name="jina-embeddings-v3"
)
collection_name = "my_collection"
vectorstore = Chroma.from_documents(collection_name=collection_name, documents=splits, embedding=embedding_function, persist_directory="./chroma_db")
#db.persist()

print("Vector store created and persisted to './chroma_db'")

Vector store created and persisted to './chroma_db'


In [25]:
# 5. Perform similarity search

query = "نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟"
search_results = vectorstore.similarity_search(query, k=2)

print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()


Top 2 most relevant chunks for the query: 'نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟'

Result 1:
Source: ./docs/NextView_Technologies_Profile_Urdu.docx
Content: کمپنی کا نام: نیکسٹ ویو ٹیکنالوجیز
قیام: 2010
مرکز: لاہور، پاکستان

Result 2:
Source: ./docs/NextView_Technologies_Profile_Urdu.docx
Content: کمپنی کا نام: نیکسٹ ویو ٹیکنالوجیز
قیام: 2010
مرکز: لاہور، پاکستان



In [47]:
# 5. Perform similarity search for second file.

query = "Please tell me website of TechNova"
search_results = vectorstore.similarity_search(query, k=2)

print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()


Top 2 most relevant chunks for the query: 'Please tell me website of TechNova'

Result 1:
Source: ./docs/TechNova_Company_Profile.txt
Content: Company Name: TechNova Solutions
Founded: 2015
Headquarters: San Francisco, California, USA

Result 2:
Source: ./docs/TechNova_Company_Profile.txt
Content: Company Name: TechNova Solutions
Founded: 2015
Headquarters: San Francisco, California, USA



In [26]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever.invoke("نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟")

[Document(metadata={'source': './docs/NextView_Technologies_Profile_Urdu.docx'}, page_content='کمپنی کا نام: نیکسٹ ویو ٹیکنالوجیز\nقیام: 2010\nمرکز: لاہور، پاکستان'),
 Document(metadata={'source': './docs/NextView_Technologies_Profile_Urdu.docx'}, page_content='کمپنی کا نام: نیکسٹ ویو ٹیکنالوجیز\nقیام: 2010\nمرکز: لاہور، پاکستان')]

In [27]:
from langchain_core.prompts import ChatPromptTemplate
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(template)

In [28]:
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt
)
rag_chain.invoke("نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟")

ChatPromptValue(messages=[HumanMessage(content="Answer the question based only on the following context:\n[Document(metadata={'source': './docs/NextView_Technologies_Profile_Urdu.docx'}, page_content='کمپنی کا نام: نیکسٹ ویو ٹیکنالوجیز\\nقیام: 2010\\nمرکز: لاہور، پاکستان'), Document(metadata={'source': './docs/NextView_Technologies_Profile_Urdu.docx'}, page_content='کمپنی کا نام: نیکسٹ ویو ٹیکنالوجیز\\nقیام: 2010\\nمرکز: لاہور، پاکستان')]\n\nQuestion: نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟\n\nAnswer: ", additional_kwargs={}, response_metadata={})])

In [29]:
def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [30]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()} | prompt
)
rag_chain.invoke("نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟")

ChatPromptValue(messages=[HumanMessage(content='Answer the question based only on the following context:\nکمپنی کا نام: نیکسٹ ویو ٹیکنالوجیز\nقیام: 2010\nمرکز: لاہور، پاکستان\n\nکمپنی کا نام: نیکسٹ ویو ٹیکنالوجیز\nقیام: 2010\nمرکز: لاہور، پاکستان\n\nQuestion: نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟\n\nAnswer: ', additional_kwargs={}, response_metadata={})])

In [31]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
question = "نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟"
response = rag_chain.invoke(question)
print(response)

2010


### Conversational RAG

#### Handling Follow Up Questions

In [32]:
# Example conversation
from langchain_core.messages import HumanMessage, AIMessage
chat_history = []
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response)
])

In [33]:
chat_history

[HumanMessage(content='نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟', additional_kwargs={}, response_metadata={}),
 AIMessage(content='2010', additional_kwargs={}, response_metadata={})]

In [34]:
from langchain_core.prompts import MessagesPlaceholder
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# history_aware_retriever = create_history_aware_retriever(
#     llm, retriever, contextualize_q_prompt
# )
contextualize_chain = contextualize_q_prompt | llm | StrOutputParser()
contextualize_chain.invoke({"input": "اس کی اہم خدمات کیا ہیں؟", "chat_history": chat_history})

'نیکسٹ ویو ٹیکنالوجیز کے لئے کچھ اہم خدمات، مثلاً:\n\n1. مواصلات کی ٹیکنالوجیز: نیکسٹ ویو موبائل مواصلات کی ٹیکنالوجیز جیسے 5G، نایاب مواد کی مواصلات، اور مواصلات کی سیکیورٹی کے لئے تحقیق اور ترقی کرتی ہے۔\n2. ڈیٹا سینٹر: نیکسٹ ویو کے ڈیٹا سینٹرز ڈیٹا کو محفوظ، سیکیور، اور ایکسرٹ کرتے ہیں۔\n3. براڈبینڈ خدمات: نیکسٹ ویو براڈبینڈ خدمات جیسے انٹرنیٹ، ٹی وی، اور فائبر اپٹک کی ترقی اور اپ گریڈیشن کے لئے کام کرتی ہے۔\n4. انٹرنیٹ آف تھنگز: نیکسٹ ویو انٹرنیٹ آف تھنگز (IoT) کی ٹیکنالوجیز، ڈیوائسز، اور سولوشنز کے لئے تحقیق اور ترقی کرتی ہے۔'

In [35]:
from langchain.chains import create_history_aware_retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)
history_aware_retriever.invoke({"input": "اس کی اہم خدمات کیا ہیں؟", "chat_history": chat_history})

[Document(metadata={'source': './docs/NextView_Technologies_Profile_Urdu.docx'}, page_content='ہمارے بارے میں:\nنیکسٹ ویو ٹیکنالوجیز پاکستان کی ایک معروف ٹیکنالوجی کمپنی ہے جو جدید سافٹ ویئر اور ڈیجیٹل حل فراہم کرتی ہے۔ \nہماری ٹیم کا مقصد کاروباروں کو ڈیجیٹل دور میں ترقی دینے کے لیے تخلیقی اور جدید ٹیکنالوجی پر مبنی خدمات فراہم کرنا ہے۔'),
 Document(metadata={'source': './docs/NextView_Technologies_Profile_Urdu.docx'}, page_content='ہمارے بارے میں:\nنیکسٹ ویو ٹیکنالوجیز پاکستان کی ایک معروف ٹیکنالوجی کمپنی ہے جو جدید سافٹ ویئر اور ڈیجیٹل حل فراہم کرتی ہے۔ \nہماری ٹیم کا مقصد کاروباروں کو ڈیجیٹل دور میں ترقی دینے کے لیے تخلیقی اور جدید ٹیکنالوجی پر مبنی خدمات فراہم کرنا ہے۔')]

In [36]:
retriever.invoke("اس کی اہم خدمات کیا ہیں؟")

[Document(metadata={'source': './docs/NextView_Technologies_Profile_Urdu.docx'}, page_content='اہم خدمات:\n1. ویب اور موبائل ایپلیکیشن ڈویلپمنٹ\n2. ڈیجیٹل مارکیٹنگ اور برانڈنگ\n3. ای کامرس سلوشنز\n4. کلاؤڈ بیسڈ ایپلیکیشنز\n5. آرٹیفیشل انٹیلیجنس پر مبنی پراڈکٹس'),
 Document(metadata={'source': './docs/NextView_Technologies_Profile_Urdu.docx'}, page_content='اہم خدمات:\n1. ویب اور موبائل ایپلیکیشن ڈویلپمنٹ\n2. ڈیجیٹل مارکیٹنگ اور برانڈنگ\n3. ای کامرس سلوشنز\n4. کلاؤڈ بیسڈ ایپلیکیشنز\n5. آرٹیفیشل انٹیلیجنس پر مبنی پراڈکٹس')]

In [37]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
    #  ("system", "Tell me joke on Programming"),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [38]:
rag_chain.invoke({"input": "اس کی اہم خدمات کیا ہیں؟", "chat_history":chat_history})

{'input': 'اس کی اہم خدمات کیا ہیں؟',
 'chat_history': [HumanMessage(content='نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2010', additional_kwargs={}, response_metadata={})],
 'context': [Document(metadata={'source': './docs/NextView_Technologies_Profile_Urdu.docx'}, page_content='ہمارے بارے میں:\nنیکسٹ ویو ٹیکنالوجیز پاکستان کی ایک معروف ٹیکنالوجی کمپنی ہے جو جدید سافٹ ویئر اور ڈیجیٹل حل فراہم کرتی ہے۔ \nہماری ٹیم کا مقصد کاروباروں کو ڈیجیٹل دور میں ترقی دینے کے لیے تخلیقی اور جدید ٹیکنالوجی پر مبنی خدمات فراہم کرنا ہے۔'),
  Document(metadata={'source': './docs/NextView_Technologies_Profile_Urdu.docx'}, page_content='ہمارے بارے میں:\nنیکسٹ ویو ٹیکنالوجیز پاکستان کی ایک معروف ٹیکنالوجی کمپنی ہے جو جدید سافٹ ویئر اور ڈیجیٹل حل فراہم کرتی ہے۔ \nہماری ٹیم کا مقصد کاروباروں کو ڈیجیٹل دور میں ترقی دینے کے لیے تخلیقی اور جدید ٹیکنالوجی پر مبنی خدمات فراہم کرنا ہے۔')],
 'answer': 'نیکسٹ ویو ٹیکنالوجیز کے اہم خدمات میں جدید ساف

### Building Multi User Chatbot

In [39]:
import sqlite3
from datetime import datetime

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
                    (id INTEGER PRIMARY KEY AUTOINCREMENT,
                     session_id TEXT,
                     user_query TEXT,
                     gpt_response TEXT,
                     model TEXT,
                     created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()

In [40]:
import uuid
session_id = str(uuid.uuid4())
chat_history = get_chat_history(session_id)
print(chat_history)
question1 = "نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟"
answer1 = rag_chain.invoke({"input": question1, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question1, answer1, "meta-llama/Llama-Vision-Free")
print(f"Human: {question1}")
print(f"AI: {answer1}\n")

[]
Human: نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟
AI: نیکسٹ ویو ٹیکنالوجیز کی بنیاد 2010 میں رکھی گئی تھی۔



In [41]:
question2 = "اس کی اہم خدمات کیا ہیں؟"
chat_history = get_chat_history(session_id)
print(chat_history)
answer2 = rag_chain.invoke({"input": question2, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question2, answer2, "meta-llama/Llama-Vision-Free")
print(f"Human: {question2}")
print(f"AI: {answer2}\n")

[{'role': 'human', 'content': 'نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟'}, {'role': 'ai', 'content': 'نیکسٹ ویو ٹیکنالوجیز کی بنیاد 2010 میں رکھی گئی تھی۔'}]
Human: اس کی اہم خدمات کیا ہیں؟
AI: نیکسٹ ویو ٹیکنالوجیز، پاکستان کی ایک معروف ٹیکنالوجی کمپنی، مختلف کاروباروں کو ڈیجیٹل دور میں ترقی دینے کے لیے تخلیقی اور جدید ٹیکنالوجی پر مبنی خدمات فراہم کرتی ہے۔

اس کی اہم خدمات میں شامل ہیں:

1- سافٹ ویئر ڈویلپمنٹ

2- ڈیجیٹل مارکیٹنگ اور ای میل مارکیٹنگ

3- ڈیٹا اینالائٹکس اور ویژن

4- اینڈلس سافٹ ویئر

5- ڈیجیٹل transformed سولوشنز

نیکسٹ ویو ٹیکنالوجیز کی خدمات کاروباروں کو ان کے ڈیجیٹل گروپ کی ترقی میں مدد فراہم کرتی ہیں تاکہ وہ زیادہ مؤثر طریقے سے اپنے ت.Σلکٹر کو سمجھیں اور چلائیں۔



New User

In [42]:
session_id = str(uuid.uuid4())
question = "What is TechNova"
chat_history = get_chat_history(session_id)
print(chat_history)
answer = rag_chain.invoke({"input": question, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question, answer, "meta-llama/Llama-Vision-Free")
print(f"Human: {question}")
print(f"AI: {answer}\n")

[]
Human: What is TechNova
AI: TechNova Solutions is a technology company founded in 2015, with its headquarters located in San Francisco, California, USA. Unfortunately, I don't have any specific information about the company's products or services, but it appears to be a relatively new player in the tech industry.

If you're looking for more information about TechNova Solutions, I suggest checking their official website or social media channels to learn more about their mission, products, and services.



In [44]:
question2 = "Plese tell me its officail website"
chat_history = get_chat_history(session_id)
print(chat_history)
answer2 = rag_chain.invoke({"input": question2, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question2, answer2, "meta-llama/Llama-Vision-Free")
print(f"Human: {question2}")
print(f"AI: {answer2}\n")

[{'role': 'human', 'content': 'What is TechNova'}, {'role': 'ai', 'content': "TechNova Solutions is a technology company founded in 2015, with its headquarters located in San Francisco, California, USA. Unfortunately, I don't have any specific information about the company's products or services, but it appears to be a relatively new player in the tech industry.\n\nIf you're looking for more information about TechNova Solutions, I suggest checking their official website or social media channels to learn more about their mission, products, and services."}, {'role': 'human', 'content': 'Plese tell me its officail website'}, {'role': 'ai', 'content': 'I made an error earlier. After conducting a search, I was unable to find a website or any online presence for a company called "TechNova Solutions" with headquarters in San Francisco, California.\n\nIt\'s possible that the company may not have a publicly available website or may be a private organization. If you have any more information abo